In [1]:
# Mount Google Drive (assuming this code is run on Google Colab)
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Change the working directory to the Deep Learning folder
%cd /content/drive/MyDrive/Deep Learning

/content/drive/MyDrive/Deep Learning


In [14]:
from numpy import zeros, ones, expand_dims, asarray
from numpy.random import randn, randint
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import Conv2D, Conv2DTranspose, Concatenate
from keras.layers import LeakyReLU, Dropout, Embedding
from keras.layers import BatchNormalization, Activation
from keras import initializers
from keras.initializers import RandomNormal
import keras.optimizers
from matplotlib import pyplot
import numpy as np
from math import sqrt
import numpy as np
from numpy.random import randn, randint
from numpy import ones, zeros
import tensorflow as tf
from matplotlib import pyplot
from math import sqrt

In [5]:
# Define the dataset directory
DATA_DIR = './animefacedataset'

In [6]:
# Set the desired size for the images (assuming square images)
image_size = 64

# Set the batch size for data loading
batch_size = 128

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define the transformation pipeline for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Rescale pixel values to the range [0, 1]
)


In [8]:
# Create the training dataset using flow_from_directory
X_train = train_datagen.flow_from_directory(
    DATA_DIR,                        # Path to the target directory
    target_size=(image_size, image_size),  # Resize images to the specified dimensions
    batch_size=batch_size,            # Number of samples per batch
    class_mode='input',
)



Found 63565 images belonging to 1 classes.


In [16]:
def generate_latent_points(latent_dim, n_samples):
    # Generate random values from a standard normal distribution
    x_input = randn(latent_dim * n_samples)

    # Reshape the 1D array of random values to a 2D array
    z_input = x_input.reshape(n_samples, latent_dim)

    # Return the generated latent points
    return z_input


In [17]:
# Function to generate real samples from a data generator
def generate_real_samples(X_train, n_samples):
    # Obtain the next batch from the generator
    X, _ = next(X_train)

    # Randomly select 'n_samples' samples from the batch
    ix = randint(0, X.shape[0], n_samples)
    X = X[ix]

    # Generate corresponding labels indicating real samples (ones)
    y = ones((n_samples, 1))

    # Return the real samples and their labels
    return X, y

# Function to generate fake samples using a generator model
def generate_fake_samples(generator, latent_dim, n_samples):
    # Generate latent points as input for the generator
    z_input = generate_latent_points(latent_dim, n_samples)

    # Generate fake images using the generator model
    images = generator.predict(z_input)

    # Generate corresponding labels indicating fake samples (zeros)
    y = zeros((n_samples, 1))

    # Return the fake samples and their labels
    return images, y


In [18]:
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    # Generate fake samples using the generator model
    _, X = generate_fake_samples(g_model, latent_dim, n_samples)

    # Rescale pixel values from the range [-1, 1] to [0, 1]
    X = (X + 1) / 2.0

    # Plot generated images in a 10x10 grid
    for i in range(100):
        pyplot.subplot(10, 10, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')  # Display grayscale images

    # Save the generator model at the current training step
    filename2 = 'anime_faces_model_%04d.h5' % (step+1)
    g_model.save(filename2)

    # Print a message indicating that the model has been saved
    print('>Saved: %s' % (filename2))


In [19]:
def save_plot(examples, n_examples):
    # Create a grid of subplots to display generated examples
    for i in range(n_examples):
        pyplot.subplot(sqrt(n_examples), sqrt(n_examples), 1 + i)

        # Turn off axis labels
        pyplot.axis('off')

        # Display the grayscale image in the subplot
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')

    # Show the plot with the generated examples
    pyplot.show()


In [20]:
from tensorflow.keras import layers, models

def define_discriminator(in_shape=(image_size, image_size, 3)):
    # Define a sequential model for the discriminator
    model = models.Sequential([
        # Convolutional layer with 64 filters, 4x4 kernel, and stride of 2x2
        layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=in_shape),
        layers.BatchNormalization(),  # Batch normalization to normalize activations
        layers.LeakyReLU(0.2),  # Leaky ReLU activation with a slope of 0.2

        # Convolutional layer with 128 filters, 4x4 kernel, and stride of 2x2
        layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        # Convolutional layer with 256 filters, 4x4 kernel, and stride of 2x2
        layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        # Convolutional layer with 512 filters, 4x4 kernel, and stride of 2x2
        layers.Conv2D(512, (4, 4), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        # Convolutional layer with 1 filter, 4x4 kernel, and stride of 1x1
        layers.Conv2D(1, (4, 4), strides=(1, 1), padding='valid'),
        layers.Flatten(),  # Flatten the output to a 1D vector
        layers.Activation('sigmoid')  # Sigmoid activation for binary classification
    ])

    # Compile the discriminator with binary cross-entropy loss and Adam optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

# Create an instance of the discriminator model
discriminator = define_discriminator()

# Display the summary of the discriminator model
discriminator.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        3136      
                                                                 
 batch_normalization (Batch  (None, 32, 32, 64)        256       
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 batch_normalization_1 (Bat  (None, 16, 16, 128)       512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0

In [24]:
from tensorflow.keras import layers, models

# Define the size of the latent vector
latent_size = 128

# Define the generator model
generator = models.Sequential([
    layers.InputLayer(input_shape=(latent_size,)),  # Input layer for the latent vector
    layers.Reshape((1, 1, latent_size)),  # Reshape the latent vector to a 4D tensor

    # Transposed convolutional layer with 512 filters, 4x4 kernel, stride of 1x1, and no bias
    layers.Conv2DTranspose(512, kernel_size=4, strides=1, padding='valid', use_bias=False),
    layers.BatchNormalization(),  # Batch normalization to normalize activations
    layers.ReLU(),  # ReLU activation

    # Transposed convolutional layer with 256 filters, 4x4 kernel, stride of 2x2, and no bias
    layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Transposed convolutional layer with 128 filters, 4x4 kernel, stride of 2x2, and no bias
    layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Transposed convolutional layer with 64 filters, 4x4 kernel, stride of 2x2, and no bias
    layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False),
    layers.BatchNormalization(),
    layers.ReLU(),

    # Transposed convolutional layer with 3 filters (for RGB channels), 4x4 kernel, stride of 2x2, no bias, and tanh activation
    layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', use_bias=False, activation='tanh')
])

# Display the summary of the generator model
generator.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 1, 1, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 4, 4, 512)         1048576   
 anspose)                                                        
                                                                 
 batch_normalization_4 (Bat  (None, 4, 4, 512)         2048      
 chNormalization)                                                
                                                                 
 re_lu (ReLU)                (None, 4, 4, 512)         0         
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 8, 8, 256)         2097152   
 Transpose)                                                      
                                                      

In [26]:
from tensorflow.keras import models
from tensorflow.keras.models import Model

def define_gan(g_model, d_model):
    # Set the discriminator to be non-trainable during GAN training
    d_model.trainable = False

    # Connect the generator to the discriminator
    gan_output = d_model(g_model.output)

    # Create a GAN model with the same input as the generator and the output from the discriminator
    model = Model(g_model.input, gan_output)

    # Compile the GAN model with binary cross-entropy loss and Adam optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

# Create an instance of the GAN model by connecting the generator and discriminator
gan_model = define_gan(generator, discriminator)


In [27]:
# Assuming X_train is a generator obtained from flow_from_directory

def train(g_model, d_model, gan_model, X_train, latent_dim, n_epochs=125, n_batch=128):
    # Calculate the number of batches per epoch
    bat_per_epo = X_train.n // n_batch
    # Calculate the total number of training steps
    n_steps = bat_per_epo * n_epochs

    # Training loop
    for i in range(n_steps):
        # Train the discriminator on real samples
        X_real, y_real = generate_real_samples(X_train, n_batch)
        d_loss_r, d_acc_r = d_model.train_on_batch(X_real, y_real)

        # Train the discriminator on fake samples generated by the generator
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
        d_loss_f, d_acc_f = d_model.train_on_batch(X_fake, y_fake)

        # Train the generator via the GAN model
        z_input = generate_latent_points(latent_dim, n_batch)
        y_gan = ones((n_batch, 1))
        g_loss, g_acc = gan_model.train_on_batch(z_input, y_gan)

        # Print the losses and accuracies for monitoring training progress
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_loss_r, d_acc_r, d_loss_f, d_acc_f, g_loss, g_acc))

        # Periodically summarize and visualize the generated samples
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, latent_dim)

    # Return the trained generator model
    return g_model


In [ ]:
latent_dim = 128
gan = train(generator, discriminator, gan_model, X_train, latent_dim, n_epochs=25, n_batch=128)

4/4 [==============================] - 0s 5ms/step
>1, dr[1.121,0.203], df[2.033,0.023], g[0.714,0.000]
4/4 [==============================] - 0s 9ms/step
>2, dr[0.000,1.000], df[0.042,1.000], g[0.690,0.695]
4/4 [==============================] - 0s 7ms/step
>3, dr[0.000,1.000], df[0.105,0.992], g[0.680,0.992]
4/4 [==============================] - 0s 7ms/step
>4, dr[0.000,1.000], df[0.014,1.000], g[0.676,1.000]
4/4 [==============================] - 0s 7ms/step
>5, dr[0.001,1.000], df[0.019,1.000], g[0.662,1.000]
4/4 [==============================] - 0s 7ms/step
>6, dr[0.001,1.000], df[0.028,1.000], g[0.660,1.000]
4/4 [==============================] - 0s 7ms/step
>7, dr[0.001,1.000], df[0.006,1.000], g[0.650,1.000]
4/4 [==============================] - 0s 7ms/step
>8, dr[0.001,1.000], df[0.015,1.000], g[0.638,1.000]
4/4 [==============================] - 0s 7ms/step
>9, dr[0.003,1.000], df[0.011,1.000], g[0.628,1.000]
4/4 [==============================] - 0s 7ms/step
>10, dr[0.001

In [ ]:
n_examples = 9
latent_points = generate_latent_points(latent_dim, n_examples)
X  = gan.predict(latent_points)
X = (X + 1) / 2.0
save_plot(X, n_examples)